In [455]:
import sys
sys.path.append('..')
import nupack as nu
import numpy as np 
from hdna import * 

In [456]:
'CGTGCTAATCGC'[::-1]

'CGCTAATCGTGC'

In [457]:
T = np.linspace(20,80,150)
f = []
for temp in T:
    M = nu.Model(ensemble='stacking', material='dna', celsius=temp)
    A = nu.Strand('GCACGATTAGCG', 'A')
    B = nu.Strand('CGTGCTAATCGC'[::-1], 'B')
    simplexstruct = nu.Structure('.'*A.__len__()+'+'+'.'*B.__len__())
    duplexstruct  = nu.Structure('('*A.__len__()+'+'+')'*B.__len__())
    C = nu.Complex([A,B], name='C')
    ca0 = 1e-6
    cb0 = 1e-6
    tube = nu.Tube(strands={A: ca0, B: cb0}, complexes=nu.SetSpec(max_size=2), name='tube1')
    tube_res = nu.tube_analysis(tubes=[tube], model=M, compute=['pairs'])
    simplex = ['B', 'A']
    duplex  = ['B+A', 'A+B']

    for q in [simplex, duplex]:
        for i, name in enumerate(q):
            q[i] = '('+name+')'
    simp = []
    dupl = []
    for complex in tube_res.tubes[tube].complex_concentrations.items():
        if complex[0].name in simplex:
            simp.append(complex)
        elif complex[0].name in duplex:
            dupl.append(complex)        

    sx = min(list(dict(simp).values()))
    dx = np.prod(list(dict(dupl).values()))
    f.append(tube_res.tubes[tube].fraction_bases_unpaired)

In [466]:
import plotly.express as px
px.line(x=T,y=f)

In [459]:
nu.structure_probability(strands=[A,B],structure=simplexstruct,model=M)

5.577626572637506e-06

In [460]:
nu.structure_probability(strands=[A,B],structure=duplexstruct,model=M)

0.04792757477831032

In [461]:
dg = nu.structure_energy(strands=[A,B], structure=duplexstruct,model=M)
ke = np.exp(-dg/(CONST.R*M.temperature))
alpha = ke*(ca0+cb0)/2

In [462]:
f1 = (0.5+np.sqrt(1+4*alpha))/(2*alpha)
f2 = (0.5-np.sqrt(1+4*ke))/(2*alpha)

In [463]:
f1 = 1 - dx/(2*sx+dx)

In [464]:
f1

0.9990084648074343